In [1]:
import os
# NOTE: material: https://drive.google.com/drive/u/0/folders/14J6oWZzULqtXfIv99064Cs0iHVdeat22

In [2]:
# NOTE: access to directory
dir_project = os.path.abspath(".")
print(f"{dir_project=}")
os.chdir(dir_project)
print(f"{os.getcwd()=}")

dir_project='/root/Documents/Projects/augmented_safeguard/D-NeRF'
os.getcwd()='/root/Documents/Projects/augmented_safeguard/D-NeRF'


In [3]:

import json
import math
import time

import cv2
import imageio
import lpips
import numpy as np
import torch
torch.autograd.set_detect_anomaly(True)
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import mcubes
import trimesh
from tqdm import tqdm, trange


In [4]:
# NOTE: extended `config_parser()`

import NeRF.config_parser

def config_parser():
    import argparse
    parser = NeRF.config_parser.config_parser()

    # NOTE: D-NeRF training options
    # ------------------------------------------
    parser.add_argument("--nerf_type", type=str, default="original", help="nerf network type")
    parser.add_argument("--N_iter", type=int, default=500000, help="num training iterations")
    parser.add_argument("--do_half_precision", action="store_true", help="do half precision training and inference")

    parser.add_argument("--add_tv_loss", action="store_true", help="evaluate tv loss")
    parser.add_argument("--tv_loss_weight", type=float, default=1.e-4, help="weight of tv loss")
    # ------------------------------------------
    
    # NOTE: D-NeRF rendering options
    # ------------------------------------------
    parser.add_argument("--not_zero_canonical", action="store_true", help="if set zero time is not the canonic space")
    parser.add_argument("--use_two_models_for_fine", action="store_true", help="use two models for fine results")
    # ------------------------------------------

    return parser

In [7]:
# NOTE: parse arguments & load config
parser = config_parser()
filename_config = "configs/mutant.txt"
configs = NeRF.config_parser.load_config(parser, filename_config)
print(configs)

{'expname': 'mutant', 'basedir': './logs', 'datadir': './data/mutant', 'dataset_type': 'blender', 'nerf_type': 'direct_temporal', 'no_batching': 'True', 'not_zero_canonical': 'False', 'use_viewdirs': 'True', 'white_bkgd': 'True', 'lrate_decay': '500', 'N_iter': '800000', 'N_samples': '64', 'N_importance': '128', 'N_rand': '500', 'testskip': '1', 'precrop_iters': '500', 'precrop_iters_time': '100000', 'precrop_frac': '0.5', 'half_res': 'True', 'do_half_precision': 'False'}


In [8]:
# NOTE: assign configs to arguments

args = parser.parse_args(args=[])
args.expname = configs['expname']
args.basedir = configs['basedir']
args.datadir = configs['datadir']
args.dataset_type = configs['dataset_type']

args.nerf_type = configs['nerf_type']
args.no_batching = configs['no_batching']
args.not_zero_canonical = configs['not_zero_canonical']

args.use_viewdirs = configs['use_viewdirs']
args.white_bkgd = configs['white_bkgd']
args.lrate_decay = int(configs['lrate_decay'])

args.N_iter = int(configs['N_iter'])
args.N_samples = int(configs['N_samples'])
args.N_importance = int(configs['N_importance'])
args.N_rand = int(configs['N_rand'])
args.testskip = int(configs['testskip'])

args.precrop_iters = int(configs['precrop_iters'])
args.precrop_iters_time = int(configs['precrop_iters_time'])
args.precrop_frac = float(configs['precrop_frac'])

args.half_res = configs['half_res']
args.do_half_precision = configs['do_half_precision']

args.no_reload = True # NOTE: for now set to True as we'll implement training code

print(args)

device = torch.device("cuda")
DEBUG=False
print(device)

torch.set_default_tensor_type('torch.cuda.FloatTensor')

Namespace(N_importance=128, N_iter=800000, N_rand=500, N_samples=64, add_tv_loss=False, basedir='./logs', chunk=32768, datadir='./data/mutant', dataset_type='blender', do_half_precision='False', expname='mutant', factor=8, ft_path=None, half_res='True', i_embed=0, i_img=500, i_print=100, i_testset=50000, i_video=50000, i_weights=10000, lindisp=False, llffhold=8, lrate=0.0005, lrate_decay=500, multires=10, multires_views=4, nerf_type='direct_temporal', netchunk=65536, netdepth=8, netdepth_fine=8, netwidth=256, netwidth_fine=256, no_batching='True', no_ndc=False, no_reload=True, not_zero_canonical='False', perturb=1.0, precrop_frac=0.5, precrop_iters=500, precrop_iters_time=100000, raw_noise_std=0.0, render_factor=0, render_only=False, render_test=False, shape='greek', spherify=False, testskip=1, tv_loss_weight=0.0001, use_two_models_for_fine=False, use_viewdirs='True', white_bkgd='True')
cuda
